## Crypro Screener and Score using BBWP and Stochastic indicators
---

### Import Libraries

To install library: !pip install *library name*

In [1]:
import ccxt
import numpy as np
import pandas as pd
import requests
from dash import Dash, dcc, Output, Input, dash_table
import dash_bootstrap_components as dbc
from dash import html
import plotly
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
import yfinance as yf
import datetime
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
import concurrent.futures
import stockstats
import ta
import time

### Enter binance API key and secret

In [2]:
# Binance API key to retrieve financial info
binance = ccxt.binance({
    'enableRateLimit': True,
    'apiKey': 'api_key_here',
    'secret': 'secret_here'
})

### Scoring system and dash app

Sync pc date and time in case of error like: *BinanceAPIException: APIError(code=-1021): Timestamp for this request was 1000ms ahead of the server's time.*

In [ ]:
start_time = time.time()

# Fetch all ticker information for trading pairs
pairs = binance.fetch_tickers()

# Specify the minimum trading volume threshold in dollars, to remove noise from the dash app
min_volume_usd = 10000000  # Set your desired volume threshold in dollars here

# Filter 'up' and 'down' pairs and stablecoins on numerator
remove_pair = []

# Filter trading pairs based on stablecoin
allow_pair = []

# Filter trading pairs based on trading volume in dollars
pairs_under_volume = []
pairs_over_volume = []

for pair, ticker in pairs.items():
    if len(pair)>11 or pair.startswith('BUSD') or pair.startswith('USDT') or pair.startswith('EUR'):
        remove_pair.append(pair)
    if pair.endswith('USDT') or pair.endswith('BUSD') or pair.endswith('ETH') or pair.endswith('BTC'):
        allow_pair.append(pair)
    if pair.endswith('USDT') or pair.endswith('BUSD') :
        if 'baseVolume' in ticker and 'last' in ticker:
            # Get the trading volume in the base currency
            volume = float(ticker['baseVolume'])

            # Get the price in the quote currency (e.g., USDT)
            price_usdt = ticker['last']

            if price_usdt is not None:
                price_usdt = float(price_usdt)
                # Calculate the trading volume in dollars
                volume_usd = volume * price_usdt

                if volume_usd < min_volume_usd:
                    pairs_under_volume.append(pair)
                    
                if volume_usd > min_volume_usd:
                    pairs_over_volume.append(pair)

binance_pairs = [x for x in pairs if x in allow_pair and x not in remove_pair]
binance_pairs = [x for x in binance_pairs if x not in pairs_under_volume]
binance_pairs = [x for x in binance_pairs if x[:3] not in [item[:3] for item in pairs_under_volume] or x[:3] in [item[:3] for item in pairs_over_volume]]
binance_pairs = [x for x in binance_pairs if ':' not in x]
binance_pairs.remove('BTC/USDT')
binance_pairs.insert(0, 'BTC/USDT')

delisted_pairs = ['BCC/BTC', 'HSR/BTC', 'DOTUP/USDT', 'MCO/ETH', 'MCO/BTC', 'SNGLS/BTC', 'BQX/BTC', 'SALT/BTC', 'MDA/BTC','DNT/ETH', 'ICN/ETH', 'YOYOW/BTC', 'SNGLS/ETH', 'BQX/ETH', 'SALT/ETH', 'MDA/ETH', 'SUB/BTC', 'SUB/ETH', 'MTH/BTC', 'MTH/ETH', 'ENG/BTC', 'ENG/ETH', 'DNT/BTC', 'ICN/BTC', 'BTG/BTC', 'BTG/ETH', 'EVX/BTC', 'EVX/ETH', 'HSR/ETH', 'YOYOW/ETH', 'MOD/BTC', 'MOD/ETH', 'VEN/BTC', 'VEN/ETH', 'RCN/BTC', 'RCN/ETH', 'DLT/BTC', 'DLT/ETH', 'BCC/ETH', 'BCC/USDT', 'BCPT/BTC', 'BCPT/ETH', 'ARN/BTC', 'ARN/ETH', 'GVT/BTC', 'GVT/ETH', 'CDT/BTC', 'CDT/ETH', 'GXS/BTC', 'GXS/ETH', 'POE/BTC', 'POE/ETH', 'QSP/BTC', 'QSP/ETH', 'XZC/BTC', 'XZC/ETH', 'TNT/BTC', 'TNT/ETH', 'FUEL/BTC', 'FUEL/ETH', 'BCD/BTC', 'BCD/ETH', 'DGD/BTC', 'DGD/ETH', 'PPT/BTC', 'PPT/ETH', 'CMT/BTC', 'CMT/ETH', 'CND/BTC', 'CND/ETH', 'LEND/BTC', 'LEND/ETH', 'WABI/BTC', 'WABI/ETH', 'TNB/BTC', 'TNB/ETH', 'GTO/BTC', 'GTO/ETH', 'OST/BTC', 'OST/ETH', 'AION/BTC', 'AION/ETH', 'BRD/BTC', 'BRD/ETH', 'EDO/BTC', 'EDO/ETH', 'NAV/BTC', 'NAV/ETH', 'TRIG/BTC', 'TRIG/ETH', 'APPC/BTC', 'APPC/ETH', 'INS/BTC', 'INS/ETH', 'CHAT/BTC', 'CHAT/ETH', 'NANO/BTC', 'NANO/ETH', 'VIA/BTC', 'VIA/ETH', 'AE/BTC', 'AE/ETH', 'RPX/BTC', 'RPX/ETH', 'NCASH/BTC', 'NCASH/ETH', 'POA/BTC', 'POA/ETH', 'WPR/BTC', 'WPR/ETH', 'QLC/BTC', 'QLC/ETH', 'GRS/BTC', 'GRS/ETH', 'CLOAK/BTC', 'CLOAK/ETH', 'GNT/BTC', 'GNT/ETH', 'BCN/BTC', 'BCN/ETH', 'REP/BTC', 'TUSD/BTC', 'TUSD/ETH', 'SKY/BTC', 'SKY/ETH', 'AGI/BTC', 'AGI/ETH', 'NXS/BTC', 'NXS/ETH', 'NPXS/BTC', 'NPXS/ETH', 'VEN/USDT', 'KEY/BTC', 'KEY/ETH', 'NAS/BTC', 'NAS/ETH', 'MFT/BTC', 'MFT/ETH', 'PHX/BTC', 'PHX/ETH', 'HC/BTC', 'HC/ETH', 'GO/BTC', 'MITH/BTC', 'BSV/BTC', 'BSV/USDT', 'USDS/USDT', 'NANO/USDT', 'MITH/USDT', 'BTCB/BTC', 'USDSB/USDT', 'GTO/USDT', 'ERD/BTC', 'ERD/USDT', 'NPXS/USDT', 'COCOS/BTC', 'MFT/USDT', 'BEAM/BTC', 'BEAM/USDT', 'HC/USDT', 'MCO/USDT', 'BULL/USDT', 'BULL/BUSD', 'BEAR/USDT', 'BEAR/BUSD', 'TCT/BTC', 'TCT/USDT', 'NANO/BUSD', 'AION/BUSD', 'AION/USDT', 'XZC/USDT', 'GXS/USDT', 'ERD/BUSD', 'LEND/USDT', 'REP/BUSD', 'REP/USDT', 'BKRW/USDT', 'BKRW/BUSD', 'LINKUP/USDT', 'DAI/BTC', 'DAI/USDT', 'DAI/BUSD', 'LEND/BUSD', 'BZRX/BTC', 'BZRX/BUSD', 'BZRX/USDT', 'TRXUP/USDT', 'SWRV/BUSD', 'LTCUP/USDT', 'NBS/BTC', 'NBS/USDT', 'HNT/BTC', 'HNT/USDT', 'EASY/ETH', 'BOT/BTC', 'BOT/BUSD', 'DNT/BUSD', 'DNT/USDT', 'HEGIC/ETH', 'HEGIC/BUSD', 'COVER/ETH', 'COVER/BUSD', 'BTCST/BTC', 'BTCST/BUSD', 'BTCST/USDT', 'USDC/BUSD', 'TUSD/BUSD', 'EASY/BTC', 'RAMP/BTC', 'RAMP/BUSD', 'RAMP/USDT', 'EPS/BTC', 'EPS/BUSD', 'EPS/USDT', 'BTG/BUSD', 'BTG/USDT', 'MIR/BTC', 'MIR/BUSD', 'MIR/USDT', 'EZ/BTC', 'EZ/ETH', 'NU/BTC', 'NU/BUSD', 'NU/USDT', 'KEEP/BTC', 'KEEP/BUSD', 'KEEP/USDT', 'HNT/BUSD', 'TRIBE/BTC', 'TRIBE/BUSD', 'TRIBE/USDT', 'USDP/BUSD', 'RGT/USDT', 'RGT/BTC', 'RGT/BUSD', 'ANY/BTC', 'ANY/BUSD', 'ANY/USDT', 'ANC/BTC', 'ANC/BUSD', 'ANC/USDT', 'GTO/BUSD']

for x in delisted_pairs:    
    if x in binance_pairs:
        binance_pairs.remove(x)

#limits are different since there will always be more candlesticks for crypto in the same time range since there are no closing hours
timeframe = pd.DataFrame({'timeframe':['4h','1d','3d','1w','1M'], 'limit':[400,400,400,600,5000], 'limit_stocks':[70,200,300,800,2000]})        
        
max_date=[]
scores = {}
badsymbol=[]
delisted=[]
NaN_values=[]

timeframes=timeframe['timeframe'].to_list()
scores_df = pd.DataFrame(index=binance_pairs, columns=timeframes)

for ticker in binance_pairs:

    ticker_date=[]
    #print(ticker)
    
    for tf in timeframes:
        

        limit = 3000

        try:
            bars = binance.fetch_ohlcv(ticker, timeframe=tf, limit=limit)
        except ccxt.BadSymbol:
            badsymbol.append(ticker)
            break
        
        df = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
               
        df.insert(0, 'datetime', pd.to_datetime(df['timestamp'], unit='ms'))
        df.insert(7, 'volume_$', df['close']*df['volume'])
        df.set_index('datetime', inplace=True)
        #print(df.index[-1]) 
        
        #check for last date to avoid delisted pairs
        if ticker in binance_pairs[0]:
            max_date.append(df.index[-1])
            
        
        ticker_date.append(df.index[-1])
        
        
                
       # elif df.index[-1] not in max_date:            
        #        binance_pairs.remove(ticker)
         #       print('removed ', ticker)
        
        # EMA
        df['ema50'] = df['close'].ewm(span=50, adjust=False).mean()
        df['ema100'] = df['close'].ewm(span=100, adjust=False).mean()

        #Stoch 14 2 5
        period = 14
        smoothing_period = 2
        signal_period = 5

        df['high_period'] = df['high'].rolling(period).max()
        df['low_period'] = df['low'].rolling(period).min()
        df['%K'] = 100 * ((df['close'] - df['low_period']) / (df['high_period'] - df['low_period'])).round(2)
        df['%D'] = df['%K'].rolling(smoothing_period).mean().rolling(signal_period).mean().round(2)
        
        if timeframes.index(tf) < 3:
         
            # BBWP
            rolling_std = df['close'].rolling(window=13).std()
            bollinger_band_width = (2 * rolling_std) / df['close'].rolling(window=13).mean()
            bbwp = bollinger_band_width.rolling(window=252).apply(lambda x: (x < x[-1]).sum() / 252 * 100)
            df['bbwp'] = bbwp
            df['bbwp_sma5'] = df['bbwp'].rolling(window=5).mean()
        
        #print(ticker,tf)

        # SCORE

        # times K crossed over 20 and crossed under 80 in the last 20 days 
        K20 = 0
        K80 = 0
        
        try:
            for i in range(-1,-20,-1):
                if df['%K'][i] > 20 and df['%K'][i-1] < 20:
                    K20 = K20 + 1

                if df['%K'][i] < 80 and df['%K'][i-1] > 80:
                    K80 = K80 + 1
        except IndexError:
            #print(ticker,'stoch error',tf)
            continue
        
        
        if df.tail(20).isna().any().any() == True:
            #print('NaN values in ticker ',ticker, tf)
            NaN_values.append(ticker)
        
        score = 0
        
        if timeframes.index(tf) < 3:

            if df['bbwp'][-1] > df['bbwp_sma5'][-1]:
                score = score + 10
                # print(ticker,tf,'+10')

            if df['bbwp'][-5:].lt(15).all():
                score = score + 10
                # print(ticker,tf,'+10')

            if df['bbwp'][-3:].lt(1).all():
                score = score + 10
                # print(ticker,tf,'+10')
            
        if df['ema50'][-1] > df['ema100'][-1]:

            score = score + 5*K20
            # print(ticker,tf,'5*',K20)

            if df['%K'][-1] > df['%D'][-1]:
                score = score + 10
                # print(ticker,tf,'+10')
                
            if df['%D'][-1] > df['%D'][-2]:
                score = score + 10
                # print(ticker,tf,'+10')

            if df['%K'][-1] < 50:
                score = score + 10
                # print(ticker,tf,'+10')

        if df['ema50'][-1] < df['ema100'][-1]:

            score = score + 5*K80
            # print(ticker,tf,'5*',K80)

            if df['%K'][-1] < df['%D'][-1]:
                score = score + 10
                # print(ticker,tf,'+10')

            if df['%D'][-1] < df['%D'][-2]:
                score = score + 10
                # print(ticker,tf,'+10')

            if df['%K'][-1] > 50:
                score = score + 10
                # print(ticker,tf,'+10')

            score = score*-1
            # print(ticker,tf,'* -1')
            
            # correction for timeframes where bbwp cannot be calculated
            if timeframes.index(tf) > 2:
                score = score*1.2885
            
        scores[(ticker,tf)] = score

        
    if all(elem not in max_date for elem in ticker_date):
        delisted.append(ticker)
        #print('removed ', ticker)
        
for ticker in delisted:
    binance_pairs.remove(ticker)
        
for (ticker, tf), score in scores.items():
    scores_df.loc[ticker, tf] = score    
    
for i in scores_df.index:    
    if i not in binance_pairs:        
        scores_df = scores_df.drop([i])


binance_pairs = [x for x in binance_pairs if x not in delisted]

scores_df.dropna(inplace=True)

scores_df['4h'] = pd.to_numeric(scores_df['4h'])
scores_df['1d'] = pd.to_numeric(scores_df['1d'])
scores_df['3d'] = pd.to_numeric(scores_df['3d'])
scores_df['1w'] = pd.to_numeric(scores_df['1w'])
scores_df['1M'] = pd.to_numeric(scores_df['1M'])

scores_df['4h SCORE'] = scores_df['4h'] / 3 + scores_df['1d'] / 3 + scores_df['3d'] / 6 + scores_df['1w'] / 6
scores_df['4h SCORE'] = np.round(scores_df['4h SCORE'], 2)
scores_df['1d SCORE'] = scores_df['1d'] / 3 + scores_df['3d'] / 3 + scores_df['1w'] / 6 + scores_df['1M'] / 6
scores_df['1d SCORE'] = np.round(scores_df['1d SCORE'], 2)



for i, j in enumerate(scores_df.index.to_list()):
    if j not in binance_pairs:
        scores_df = scores_df.drop(j)
        
scores_df.insert(0, 'Pair', scores_df.index)

trend_dict = {}

trend_ = ['Bull','Bear']
score_top = ['4h SCORE', '1d SCORE']

for y in score_top:
    for x in trend_:
        if x == 'Bull':
            df_trend = scores_df.sort_values(y, ascending=False).head(20)
        if x == 'Bear':
            df_trend = scores_df.sort_values(y, ascending=False).tail(20).iloc[::-1]      
        trend_dict[(y,x)] = df_trend
    df_trend = df_trend.sort_values(y, ascending= False)

    
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")
    
# Plotly app
    
# Colors
# Custom color styles
CHART_COLOR = 'rgba(0, 30, 35, 1)'
GRID_COLOR = 'rgba(60, 100, 100, 1)'
BACKGROUND_COLOR = 'rgba(0, 45, 45, 1)'
TEXT_COLOR = 'rgba(150, 200, 200, 1)'

# For dropdown3
trend = ['Bull','Bear']
score_top = ['4h SCORE', '1d SCORE']

# components
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])
mytitle = dcc.Markdown(children='## CRYPTO SCREENER',style={'margin-bottom': '10px', 'margin-top': '10px','color':TEXT_COLOR})
mygraph = dcc.Graph(id='mygraph',figure={})
dropdown1 = dcc.Dropdown(id = 'mydropdown1',
                        options=[{'label': c, 'value': c} for c in binance_pairs],
                        value='BTC/USDT',  
                        clearable=True)
dropdown2 = dcc.Dropdown(id = 'mydropdown2',
                        options=[{'label': c, 'value': c} for c in timeframe['timeframe']],
                        value='4h',  
                        style={'margin-bottom': '20px'},
                        clearable=True)
dropdown3 = dcc.Dropdown(id = 'mydropdown3',
                        options=[{'label': c, 'value': c} for c in trend],
                        value='Bull',  
                        clearable=True)
dropdown4 = dcc.Dropdown(id = 'mydropdown4',
                        options=[{'label': c, 'value': c} for c in score_top],
                        value='4h SCORE',  
                        clearable=True)
table = dbc.Table.from_dataframe(scores_df, id='table', bordered=True)


app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([mytitle])        
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Row([                
                dbc.Col(dropdown3,style={'margin-bottom': '120px'}),
                dbc.Col(dropdown4,style={'margin-bottom': '120px'})
            ]),
            dbc.Row(table)
        ],width=3),
        dbc.Col([
            dbc.Row([
                dbc.Col(dropdown1, width=4, style={'marginLeft': '0px'}),
                dbc.Col(dropdown2, width=4)
            ]),
            dbc.Row(mygraph, style={'marginLeft': '0px'})
        ], width=4)
        
    ]),    
],  
    fluid = True,
    style={
    
    'marginLeft': '0px',
    'backgroundColor': BACKGROUND_COLOR,
    'height': '969px'
    }
)

# Callback
@app.callback(
    [Output(component_id='table', component_property='children')],
    [Input(component_id='mydropdown3', component_property='value'),
    Input(component_id='mydropdown4', component_property='value')]
)

def update_table(user_input3, user_input4):
    
       
    table_final = dbc.Table.from_dataframe(trend_dict[user_input4, user_input3], id='table', bordered=True,
                                 style={'color': TEXT_COLOR,'borderColor': GRID_COLOR,'backgroundColor':CHART_COLOR,'fontSize':'12px','line-height':'5px'})
    
    return [table_final]

# Callback
@app.callback(
    [Output(component_id='mygraph', component_property='figure')],
    [Input(component_id='mydropdown1', component_property='value'),
    Input(component_id='mydropdown2', component_property='value')]
)

        
def update_graph(user_input1,user_input2): 
    
    # Crypto
    
    if user_input1 in binance_pairs:
        
        bars = binance.fetch_ohlcv(user_input1, timeframe=user_input2, limit=timeframe.loc[timeframe['timeframe']==user_input2, 'limit'].iloc[0])
        df = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
        df.insert(0, 'datetime', pd.to_datetime(df['timestamp'], unit='ms'))
        df.set_index('datetime', inplace=True)


    # Stocks    
#     if user_input1 in stocks:
        
#         if user_input2 == '4h':   # API does not retrieve 4h info, we need to turn 1h into 4h
#             print('4h')
#             df = yf.download(user_input1, start=date.today() - timedelta(days=int(timeframe.loc[timeframe['timeframe']==user_input2, 'limit_stocks'].iloc[0])), end=date.today(), interval = '1h')
#             df.columns= df.columns.str.lower()
#             df = stockstats.StockDataFrame.retype(df)
            
            
#             # Resample to 4-hour intervals, if the offset wasn't given, we would end up with candles starting at 8am and 12pm
#             df = df.resample('4H',offset='1H30T').agg({
#                 'open': 'first',
#                 'high': 'max',
#                 'low': 'min',
#                 'close': 'last',
#                 'volume': 'sum'
#             })
#             df.dropna(axis=0, inplace=True)
            
#         elif user_input2 == '3d': # API does not retrieve 3d info, we need to turn 1d into 3d
#             print('3d')
#             df = yf.download(user_input1, start=date.today() - timedelta(days=int(timeframe.loc[timeframe['timeframe']==user_input2, 'limit_stocks'].iloc[0])), end=date.today(), interval = '1d')
#             df.columns= df.columns.str.lower()
#             df = stockstats.StockDataFrame.retype(df)
            
#             # Resample the data using the business day frequency
#             df = df.resample('3B').agg({
#                 'open': 'first',
#                 'high': 'max',
#                 'low': 'min',
#                 'close': 'last',
#                 'volume': 'sum'
#             })
#             df.dropna(axis=0, inplace=True)
#             df.head()
            
#         elif user_input2 == '1w':  # the nomenclature is different from the crypto API, 1w vs 1wk
            
#             print('1w')
#             df = yf.download(user_input1, start=date.today() - timedelta(days=int(timeframe.loc[timeframe['timeframe']==user_input2, 'limit_stocks'].iloc[0])), end=date.today(), interval = '1wk')
#             df.columns= df.columns.str.lower()
#             df = stockstats.StockDataFrame.retype(df)
        
#         else:
#             print('1d')
#             df = yf.download(user_input1, start=date.today() - timedelta(days=int(timeframe.loc[timeframe['timeframe']==user_input2, 'limit_stocks'].iloc[0])), end=date.today(), interval = user_input2)
#             df.columns= df.columns.str.lower()
#             df = stockstats.StockDataFrame.retype(df)
            
    
    #EMA
    df['ema50'] = df['close'].ewm(span=50, adjust=False).mean()
    df['ema100'] = df['close'].ewm(span=100, adjust=False).mean()
            
    #Stoch 14 2 5
    period = 14
    smoothing_period = 2
    signal_period = 5

    df['high_period'] = df['high'].rolling(period).max()
    df['low_period'] = df['low'].rolling(period).min()
    df['%K'] = 100 * ((df['close'] - df['low_period']) / (df['high_period'] - df['low_period'])).round(2)
    df['%D'] = df['%K'].rolling(smoothing_period).mean().rolling(signal_period).mean().round(2)
    
    # BBWP
    rolling_std = df['close'].rolling(window=13).std()
    bollinger_band_width = (2 * rolling_std) / df['close'].rolling(window=13).mean()
    bbwp = bollinger_band_width.rolling(window=252).apply(lambda x: (x < x[-1]).sum() / 252 * 100)
    df['bbwp'] = bbwp
    df['bbwp_sma5'] = df['bbwp'].rolling(window=5).mean()
    # Create the grid of subplots
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[5, 1, 1])
    
    
    #Plot EMA
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['ema50'],
            mode='lines',
            line=dict(color='Gold', width=1),
            name='EMA 50'
        ),
        row=1, col=1
        )
    
    # Update the layout to change the font color of the legend label
    fig.update_layout(
        legend_font=dict(color=TEXT_COLOR)
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['ema100'],
            mode='lines',
            line=dict(color='rgba(250, 140, 20, 1)', width=1),
            name='EMA 100',
            textfont=dict(
                color=TEXT_COLOR
    )),
        row=1, col=1
        )
    
    fig.update_traces(textfont_color=TEXT_COLOR)
        
    # Create the candles
    fig.add_trace(
        go.Candlestick(
            x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            increasing=dict(line=dict(color='rgba(69, 209, 14, 1)')),
            decreasing=dict(line=dict(color='red')),
            showlegend=False,
            name='price'
        ),
        row=1, col=1
        )
    
    # Remove the white line at y=0
    fig.update_yaxes(zeroline=False, row=1, col=1)
    
    #title
    fig.update_layout(
    title={
        'text': f'<h1>{user_input1}</h1>',
        },
    title_font={
        'size': 70,
        'color': TEXT_COLOR
        },
    )
            
    # Remove rangeslider and add size
    fig.update_layout(
        title={
        'text': f"{user_input1} ",
        'x': 0.47,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font={
        'size': 20
    },
        xaxis=dict(
            rangeslider=dict(visible=False),
            type='date'
        ),
    margin=dict(l=0),  
    height=650, 
    width=1400,
    plot_bgcolor=CHART_COLOR,
    paper_bgcolor=BACKGROUND_COLOR
    )

    #Plot BBWP
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['bbwp'],
            mode='lines',
            line=dict(color='rgba(92, 222, 198, 1)', width=1),
            name='BBWP',
            showlegend=False),
            row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['bbwp_sma5'],
            mode='lines',
            line=dict(color='rgba(230, 200, 180, 1)', width=0.7),
            name='BBWP sma5',
            showlegend=False),
            row=2, col=1
    )
    if (df['bbwp'] >= 98).any():
        bbwp_100 = df.index[df['bbwp'] >= 98]
        for idx in bbwp_100:
            fig.add_shape(
            type="rect",
                x0=idx,
                y0=0,
                x1=idx,
                y1=100,
                fillcolor='rgba(239, 58, 119, 1)',
                opacity=1,
                layer="above",
                line=dict(color='rgba(239, 58, 119, 1)', width=1.2),
                row=2, col=1
            )
            
    if (df['bbwp'] <= 2).any():
        bbwp_0 = df.index[df['bbwp'] <= 2]
        for idx in bbwp_0:
            fig.add_shape(
            type="rect",
                x0=idx,
                y0=0,
                x1=idx,
                y1=100,
                fillcolor='rgba(37, 155, 240, 1)',
                opacity=1,
                layer="above",
                line=dict(color='rgba(37, 155, 240, 1)', width=1.2),
                row=2, col=1
            )
            
    # Remove the white line at y=0 and set range and font size
    fig.update_yaxes(zeroline=False, range=[0, 100], tickfont=dict(size=10), row=2, col=1)


    #Plot Stoch
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['%K'],
            mode='lines',
            line=dict(color='rgba(0, 210, 255, 1)', width=1),
            name='%K',
            showlegend=False),
            row=3, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df['%D'],
            mode='lines',
            line=dict(color='indianred', width=1),
            name='%D',
            showlegend=False),
            row=3, col=1
    )
    

    
    # Stoch = 80
    fig.add_trace(
    go.Scatter(
        x=[df.index[0], df.index[-1]],
        y=[80, 80],
        mode='lines',
        line=dict(color='grey', width=0.7, dash='dash'),
        showlegend=False,
        name='stoch80'),
    row=3, col=1
    )
    # Stoch = 20
    fig.add_trace(
    go.Scatter(
        x=[df.index[0], df.index[-1]],
        y=[20, 20],
        mode='lines',
        line=dict(color='grey', width=0.7, dash='dash'),
        showlegend=False,
        name='stoch20'),
    row=3, col=1
    )
    
    # Remove the white line at y=0 and set range and font size
    fig.update_yaxes(zeroline=False, range=[0, 100], tickfont=dict(size=10), row=3, col=1)

    fig.update_yaxes(showgrid=True, gridwidth=1, tickfont_color=TEXT_COLOR, linecolor=GRID_COLOR, gridcolor=GRID_COLOR, layer='below traces', showline=False, row=1, col=1)
    fig.update_xaxes(showgrid=True, gridwidth=1, tickfont_color=TEXT_COLOR, linecolor=GRID_COLOR, gridcolor=GRID_COLOR, layer='below traces', showline=False, row=1, col=1)
    fig.update_yaxes(showgrid=True, gridwidth=1, tickfont_color=TEXT_COLOR, linecolor=GRID_COLOR, gridcolor=GRID_COLOR, title_text='BBWP', layer='below traces', showline=False, title_font=dict(color=TEXT_COLOR), row=2, col=1)
    fig.update_xaxes(showgrid=True, gridwidth=1, tickfont_color=TEXT_COLOR, linecolor=GRID_COLOR, gridcolor=GRID_COLOR, layer='below traces', showline=False, row=2, col=1)
    fig.update_yaxes(showgrid=True, gridwidth=1, tickfont_color=TEXT_COLOR, linecolor=GRID_COLOR, gridcolor=GRID_COLOR, title_text='STOCH', layer='below traces', showline=False, title_font=dict(color=TEXT_COLOR), row=3, col=1)
    fig.update_xaxes(showgrid=True, gridwidth=1, tickfont_color=TEXT_COLOR, linecolor=GRID_COLOR, gridcolor=GRID_COLOR, layer='below traces', row=3, showline=False, col=1)
    


    
#     if user_input1 in stocks:
    
#         #Hide weekends and holidays (for 1d and 3d)
#         if user_input2 == '1d' or user_input2 == '3d':
#             fig.update_xaxes(
#             rangebreaks=[
#                 { 'pattern': 'day of week', 'bounds': [6, 1]},
#                 dict(values = ['2023-01-16',
#                                                '2023-02-20',
#                                                '2023-04-07',
#                                                '2023-05-29',
#                                                '2023-06-19',
#                                                '2023-07-04',
#                                                '2023-09-04',
#                                                '2023-11-23',
#                                                '2023-12-25'])
#             ]
#         )
    
#         #Hide weekends, holidays and non-trading hours (for 4h)
#         if user_input2 == '4h':
#             fig.update_xaxes(
#             rangebreaks=[
#                 { 'pattern': 'day of week', 'bounds': [6, 1]},
#                 { 'pattern': 'hour', 'bounds':[16,9.5]},
#                 dict(values = ['2023-01-16 09:30:00-05:30', #'2023-01-16 10:30:00-04:00', '2023-01-16 11:30:00-04:00', '2023-01-16 12:30:00-04:00', '2023-01-16 13:30:00-04:00', '2023-01-16 14:30:00-04:00', '2023-01-16 15:30:00-04:00',
#                                '2023-02-20 09:30:00-05:30',#'2023-02-20 10:30:00-04:00','2023-02-20 11:30:00-04:00','2023-02-20 12:30:00-04:00','2023-02-20 13:30:00-04:00','2023-02-20 14:30:00-04:00','2023-02-20 15:30:00-04:00',
#                                '2023-04-07 09:30:00-05:30',#'2023-04-07 10:30:00-04:00','2023-04-07 11:30:00-04:00','2023-04-07 12:30:00-04:00','2023-04-07 13:30:00-04:00','2023-04-07 14:30:00-04:00','2023-04-07 15:30:00-04:00', 
#                                '2023-05-29 09:30:00-05:30',#'2023-05-29 10:30:00-04:00','2023-05-29 11:30:00-04:00','2023-05-29 12:30:00-04:00','2023-05-29 13:30:00-04:00','2023-05-29 14:30:00-04:00','2023-05-29 15:30:00-04:00',                           
#                                '2023-06-19 09:30:00-05:30',#'2023-06-19 10:30:00-04:00','2023-06-19 11:30:00-04:00','2023-06-19 12:30:00-04:00','2023-06-19 13:30:00-04:00','2023-06-19 14:30:00-04:00','2023-06-19 15:30:00-04:00',
#                                '2023-07-04 09:30:00-05:30',#'2023-07-04 10:30:00-04:00','2023-07-04 11:30:00-04:00','2023-07-04 12:30:00-04:00','2023-07-04 13:30:00-04:00','2023-07-04 14:30:00-04:00','2023-07-04 15:30:00-04:00',
#                                '2023-09-04 09:30:00-05:30',#'2023-09-04 10:30:00-04:00','2023-09-04 11:30:00-04:00','2023-09-04 12:30:00-04:00','2023-09-04 13:30:00-04:00','2023-09-04 14:30:00-04:00','2023-09-04 15:30:00-04:00',
#                                '2023-11-23 09:30:00-05:30',#'2023-11-23 10:30:00-04:00','2023-11-23 11:30:00-04:00','2023-11-23 12:30:00-04:00','2023-11-23 13:30:00-04:00','2023-11-23 14:30:00-04:00','2023-11-23 15:30:00-04:00',
#                                '2023-12-25 09:30:00-05:30' #'2023-12-25 10:30:00-04:00','2023-12-25 11:30:00-04:00','2023-12-25 12:30:00-04:00','2023-12-25 13:30:00-04:00','2023-12-25 14:30:00-04:00','2023-12-25 15:30:00-04:00'       
#                      ])])
    

    
    
    #show plot and markdowns
    return [fig]

# Run app
if __name__=='__main__':
    app.run_server(port=8040)

Elapsed time: 223.70691013336182 seconds
Dash is running on http://127.0.0.1:8040/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8040
Press CTRL+C to quit
127.0.0.1 - - [12/Jun/2023 20:48:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 20:48:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 20:48:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 20:48:42] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 20:48:42] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 20:48:42] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 20:48:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 20:48:42] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 20:48:42] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [12/Jun/2023 20:48:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/20